https://www.kaggle.com/c/nbme-score-clinical-patient-notes/data

https://huggingface.co/docs/transformers/training

https://github.com/huggingface/notebooks/blob/master/transformers_doc/training.ipynb

In [1]:
# !mv kaggle.json /root/.kaggle/

In [2]:
# !kaggle competitions download -c nbme-score-clinical-patient-notes

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# mv /content/datasets/nbme-score-clinical-patient-notes.zip /content/drive/MyDrive

In [7]:
cd /content/drive/MyDrive/datasets/clinical-patient-notes/

/content/drive/MyDrive/datasets/clinical-patient-notes


In [ ]:
# pwd

'/content'

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# ls 

In [9]:
# !unzip nbme-score-clinical-patient-notes.zip

In [10]:
# pip install pandas

In [11]:
import pandas as pd

In [12]:
# features = pd.read_csv('features.csv')
# patient_notes = pd.read_csv('patient_notes.csv')
# test = pd.read_csv('test.csv')
# train = pd.read_csv('train.csv')

TODO:  
follow the tutorial for fine-tuning a pretrained model using clinical patient notes dataset

In [13]:
pip install transformers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 6.5 MB 30.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [15]:
# from torch.utils.data import Dataset
# class JsonDataset(Dataset):
#     def __init__(self, csv_file, max_length=64):
#         _dataset = pd.read_csv(csv_file)
#         conversations = []
#         self.speaker1 = _dataset['0']
#         self.speaker2 = _dataset['1']
#         self.tokenized_speaker1 = [tokenizer(m, padding="max_length", max_length=max_length, truncation=True) for m in self.speaker1.values]
#         self.tokenized_speaker2 = [tokenizer(m, padding="max_length", max_length=max_length, truncation=True) for m in self.speaker2.values]#tokenizer([m for m in speaker2.values], padding="max_length", truncation=True)
#         #print(full_conversations)
        
        
#         # self.dataset=full_conversations
#     def __len__(self):
#         return len(self.tokenized_speaker1)

#     def __getitem__(self, idx):
#         question = self.tokenized_speaker1[idx]#tokenizer(speaker1[idx], padding="max_length", truncation=True)
#         answer = self.tokenized_speaker2[idx]#tokenizer(speaker2[idx], padding="max_length", truncation=True)
        
#         return torch.Tensor(question['input_ids']).long(),torch.Tensor(answer['input_ids']).long()
    
# #[m for m in full_train_dataset['messages']]


In [16]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, csv_file, max_length=64, transform=None):
        dataset = pd.read_csv(csv_file)
        self.annotation = dataset['annotation']
        self.feature = dataset['feature_num']
        self.tokenized_annotation = [tokenizer(m, padding="max_length", max_length=max_length, truncation=True) for m in self.annotation.values]
#         self.tokenized_feature = [tokenizer(m, padding="max_length", max_length=max_length, truncation=True) for m in self.feature.values]

  
    def __len__(self):
        return len(self.tokenized_annotation)

    def __getitem__(self, idx):
        annotation = self.tokenized_annotation[idx]
#         print(annotation)
#         feature = self.tokenized_feature[idx]
        feature = self.feature[idx]
#         print(feature)
        
#        return torch.Tensor(annotation['input_ids']).long(),torch.Tensor(feature).long()
        return torch.Tensor(annotation['input_ids']).long(),feature


In [17]:
#from torchvision.transforms import Resize
#dataset=MyDataset(csv_file="/home/cassidy/datasets/nbme_clinical_patient_notes/train.csv", transform=Resize())
train_dataset=MyDataset(csv_file="/content/drive/MyDrive/datasets/clinical-patient-notes/train.csv")

In [18]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)

# for X, y in train_dataloader:
#     print("Shape of X [N, C, H, W]: ", X.shape)
#     print("Shape of y: ", y.shape, y.dtype)
#     break

In [19]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=917)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [20]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

TODO: 

transfer to Google Collab
continue following the fine-tune a pretrained model tutorial using the clinical notes dataset

In [21]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [22]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

TODO: print out batch from example using example dataset and compare to my batch and see what k/v pairs are

https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification.forward.labels

In [31]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
#add description of loss

iteration=0
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch={"input_ids":batch[0].to(device), "labels":batch[1].to(device)}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        progress_bar.set_description(f"Loss {loss.item():.5f}")

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        if iteration%500==0:
          torch.save(model.state_dict(),"/content/drive/MyDrive/datasets/clinical-patient-notes/saved_models/saved_model_%d"%iteration)

        iteration+=1


  0%|          | 0/5364 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

1) let the model run 2) save the model 3) submit to kaggle


TODO: train model on cosmx dataset, search online to see which model works best
https://nanostring.com/products/cosmx-spatial-molecular-imager/ffpe-dataset/

In [25]:
if iteration%10_000==0:
  torch.save(model.state_dict(),"path to save model"+{iteration})

NameError: ignored

In [32]:
print(iteration)

348
